In [1]:
# # Geração de Insumos Hidrológicos Básicos (Pré-Análise)
#
# Este notebook gera os rasters de **Direção de Fluxo**, **Acumulação de Fluxo** e um vetor da **Rede de Drenagem** para **toda a área do MDE**.
#
# **Objetivo:** Criar os arquivos necessários para carregar em um software SIG (como o QGIS) e permitir a **identificação visual** de um ponto de exutório.
#
# **Arquivos Gerados:**
# 1. `flow_direction.tif`
# 2. `flow_accumulation.tif`
# 3. `rede_drenagem_total.geojson`

# ----------------------------------------------------------------------
# --- CÉLULA 0: Configuração e Bibliotecas Globais ---
# ----------------------------------------------------------------------
import rasterio
from rasterio import features
import geopandas as gpd
from pyproj import CRS
import osmnx as ox
import numpy as np
from shapely.geometry import box, shape, LineString
import warnings
import os


try:
    from pysheds.grid import Grid
except ImportError:
    print("Erro: Biblioteca 'pysheds' não encontrada. Instale com 'pip install pysheds'")
    raise

# Ignorar warnings futuros
warnings.filterwarnings("ignore", category=FutureWarning)
("Bibliotecas globais importadas com sucesso.")

C:\Users\G15-PHCCG\miniconda3\envs\hydro_mde\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


'Bibliotecas globais importadas com sucesso.'

## Parâmetros iniciais

In [2]:
# ----------------------------------------------------------------------
# --- CÉLULA 1: Parâmetros Globais e Seletor de Cenário ---
# ----------------------------------------------------------------------
# Ajuste estes caminhos e valores conforme necessário

# [ENTRADA] Caminho para o seu Modelo Digital de Elevação (MDE/DEM)
INPUT_MDE_PATH = r'srtm_data.tif'

# [SAÍDA] Diretório para salvar os resultados
OUTPUT_DIR = 'resultados_hidrologia_basica'

# Cria o diretório de saída se não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Diretório de saída: '{OUTPUT_DIR}'")

# ==============================================================================
# --- PARÂMETROS HIDROLÓGICOS GLOBAIS (AJUSTÁVEIS PELO ANALISTA) ---
# ==============================================================================

# --- 1. Profundidade do Canal (CHANNEL_DEPTH) ---
# Este é o parâmetro "gatilho" do modelo.
# Ele define a altura (em metros) que o rio sobe.
#
# GUIA DE CALIBRAÇÃO (Brumadinho, Rio Paraopeba):
# - 10.0 metros: Valor que se alinha aos eventos severos de cheia
#                observados em 2022, que por sua vez são
#                classificados como próximos a um TR de 100 Anos.
# -  7.5 metros: (Hipotético) Simula um evento de TR 50 Anos.
# -  4.0 metros: (Hipotético) Simula um evento de TR 10 Anos.
#
CHANNEL_DEPTH = 10.0 # (Padrão: Calibrado para TR 100 Anos)

# --- 2. Limiar de Drenagem (STREAM_THRESHOLD) ---
# Define a área mínima (em células) para formar um "rio".
STREAM_THRESHOLD = 1000

print(f"\n--- Cenário de Simulação Ativado ---")
print(f"Limiar de Drenagem (STREAM_THRESHOLD): {STREAM_THRESHOLD} células")
print(f"Profundidade do Canal (CHANNEL_DEPTH): {CHANNEL_DEPTH} metros")
print(f"(Nota: Profundidade calibrada para simular o evento de TR 100 Anos)")
print("-------------------------------------")

Diretório de saída: 'resultados_hidrologia_basica'

--- Cenário de Simulação Ativado ---
Limiar de Drenagem (STREAM_THRESHOLD): 1000 células
Profundidade do Canal (CHANNEL_DEPTH): 10.0 metros
(Nota: Profundidade calibrada para simular o evento de TR 100 Anos)
-------------------------------------


## Dados auxiliares : Flow direction + Flow acumulation

In [3]:
# ----------------------------------------------------------------------
# --- CÉLULA 2: Cálculo de Direção e Acumulação de Fluxo ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 2: Direção e Acumulação de Fluxo ---")

try:
    if not os.path.exists(INPUT_MDE_PATH):
        raise FileNotFoundError(f"Arquivo MDE '{INPUT_MDE_PATH}' não encontrado.")

    print("Carregando MDE e instanciando Grid PySheds...")
    grid = Grid.from_raster(INPUT_MDE_PATH, data_name='dem')
    dem = grid.read_raster(INPUT_MDE_PATH)

    # Condicionamento hidrológico
    print("Condicionando MDE (fill_pits, fill_depressions, resolve_flats)... (Pode demorar)")
    pit_filled_dem = grid.fill_pits(dem)
    flooded_dem = grid.fill_depressions(pit_filled_dem)
    inflated_dem = grid.resolve_flats(flooded_dem) # MDE condicionado

    # Direção e acumulação do fluxo
    print("Calculando direção e acumulação do fluxo... (Pode demorar)")
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
    acc = grid.accumulation(fdir, dirmap=dirmap)

    # Calcular o aspect com NumPy
    print("Calculando o aspect do terreno com NumPy...")
    gy, gx = np.gradient(inflated_dem)
    aspect_rad = np.arctan2(gy, -gx)
    aspect_deg = np.degrees(aspect_rad)
    aspect = (aspect_deg + 360) % 360
    aspect[((gx == 0) & (gy == 0))] = -1 # Marcar áreas planas com -1

    # Salvar rasters intermediários
    fdir_path = os.path.join(OUTPUT_DIR, 'flow_direction.tif')
    acc_path = os.path.join(OUTPUT_DIR, 'flow_accumulation.tif')
    aspect_path = os.path.join(OUTPUT_DIR, 'aspect.tif')

    print(f"Salvando raster de direção de fluxo em '{fdir_path}'...")
    grid.to_raster(fdir, fdir_path, nodata=-9999)

    print(f"Salvando raster de acumulação de fluxo em '{acc_path}'...")
    grid.to_raster(acc, acc_path, nodata=-9999)

    print(f"Salvando raster de aspect em '{aspect_path}'...")
    grid.to_raster(aspect, aspect_path, nodata=-1) # Usar -1 como nodata para o aspect

    print("Célula 2 concluída com sucesso!")
    print("Variáveis `grid`, `fdir`, `acc`, `dirmap`, `inflated_dem` disponíveis para a próxima célula.")

except Exception as e:
    print(f"Erro inesperado na Célula 2: {e}")
    raise



--- Iniciando Célula 2: Direção e Acumulação de Fluxo ---
Carregando MDE e instanciando Grid PySheds...
Condicionando MDE (fill_pits, fill_depressions, resolve_flats)... (Pode demorar)
Calculando direção e acumulação do fluxo... (Pode demorar)
Calculando o aspect do terreno com NumPy...
Salvando raster de direção de fluxo em 'resultados_hidrologia_basica\flow_direction.tif'...
Salvando raster de acumulação de fluxo em 'resultados_hidrologia_basica\flow_accumulation.tif'...
Salvando raster de aspect em 'resultados_hidrologia_basica\aspect.tif'...
Célula 2 concluída com sucesso!
Variáveis `grid`, `fdir`, `acc`, `dirmap`, `inflated_dem` disponíveis para a próxima célula.


## Dados auxiliares : Declividade

In [4]:
# ----------------------------------------------------------------------
# --- CÉLULA 2.5: Cálculo de Declividade (Slope) ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 2.5: Cálculo de Declividade (Slope) ---")

# Verificar se a variável da Célula 2 existe
if 'inflated_dem' not in locals():
    print("Erro: Variável 'inflated_dem' da Célula 2 não encontrada. Execute a Célula 2 primeiro.")
    raise NameError("Variável 'inflated_dem' ausente.")

try:
    print("Calculando a declividade (slope) do terreno em graus...")
    # Resolução do raster (tamanho do pixel)
    x_res = grid.viewfinder.affine[0]
    y_res = -grid.viewfinder.affine[4] # É negativo na affine

    # Gradiente
    gy, gx = np.gradient(inflated_dem, y_res, x_res)

    # Magnitude do gradiente
    slope_rad = np.arctan(np.sqrt(gx**2 + gy**2))
    slope_deg = np.degrees(slope_rad)

    # Salvar o raster de declividade
    slope_path = os.path.join(OUTPUT_DIR, 'slope.tif')
    print(f"Salvando raster de declividade em '{slope_path}'...")
    grid.to_raster(slope_deg, slope_path, nodata=-9999)

    print("Célula 2.5 concluída com sucesso!")

except Exception as e:
    print(f"Erro inesperado na Célula 2.5: {e}")
    raise



--- Iniciando Célula 2.5: Cálculo de Declividade (Slope) ---
Calculando a declividade (slope) do terreno em graus...
Salvando raster de declividade em 'resultados_hidrologia_basica\slope.tif'...
Célula 2.5 concluída com sucesso!


## Dados auxiliares :  índice de Umidade Topográfica

In [5]:
# ----------------------------------------------------------------------
# --- CÉLULA 2.6: Cálculo do Índice de Umidade Topográfica (TWI) ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 2.6: Cálculo do Índice de Umidade Topográfica (TWI) ---")

# Verificar se as variáveis necessárias existem
if 'acc' not in locals() or 'inflated_dem' not in locals() or 'grid' not in locals():
    print("Erro: Variáveis da Célula 2 não encontradas. Execute a Célula 2 primeiro.")
    raise NameError("Variáveis da Célula 2 ausentes.")

try:
    print("Calculando o Índice de Umidade Topográfica (TWI)...")
    # Resolução do raster (tamanho do pixel)
    x_res = grid.viewfinder.affine[0]
    y_res = -grid.viewfinder.affine[4]
    cell_area = x_res * y_res

    # Recalcular declividade em radianos
    gy, gx = np.gradient(inflated_dem, y_res, x_res)
    slope_rad = np.arctan(np.sqrt(gx**2 + gy**2))

    # Evitar divisão por zero em áreas planas, substituindo por um valor pequeno
    slope_rad[slope_rad == 0] = 0.001

    # Área de contribuição específica (Specific Catchment Area)
    sca = (acc + 1) * cell_area

    # TWI
    twi = np.log(sca / np.tan(slope_rad))

    # Salvar o raster TWI
    twi_path = os.path.join(OUTPUT_DIR, 'twi.tif')
    print(f"Salvando raster TWI em '{twi_path}'...")
    grid.to_raster(twi, twi_path, nodata=-9999)

    print("Célula 2.6 concluída com sucesso!")

except Exception as e:
    print(f"Erro inesperado na Célula 2.6: {e}")
    raise



--- Iniciando Célula 2.6: Cálculo do Índice de Umidade Topográfica (TWI) ---
Calculando o Índice de Umidade Topográfica (TWI)...
Salvando raster TWI em 'resultados_hidrologia_basica\twi.tif'...
Célula 2.6 concluída com sucesso!


## Dados auxiliares :  Rede de drenagem

In [6]:
# ----------------------------------------------------------------------
# --- CÉLULA 3: Vetorização da Rede de Drenagem (Toda a Área) ---
# ----------------------------------------------------------------------
print("\n--- Iniciando Célula 3: Vetorização da Rede de Drenagem ---")

# Verificar se as variáveis da Célula 2 existem
if 'grid' not in locals() or 'fdir' not in locals() or 'acc' not in locals() or 'dirmap' not in locals():
    print("Erro: Variáveis da Célula 2 não encontradas. Execute a Célula 2 primeiro.")
    raise NameError("Variáveis da Célula 2 ausentes.")

try:
    print(f"Criando máscara de rios com limiar > {STREAM_THRESHOLD}...")
    # 1. Máscara de rios (baseada no limiar global)
    streams_mask = acc > STREAM_THRESHOLD

    # 2. EXTRAÇÃO DA REDE VETORIAL
    print("Extraindo a rede de drenagem como feições vetoriais... (Pode demorar)")
    # Nota: Usamos fdir e streams_mask de toda a área, não uma versão recortada
    network = grid.extract_river_network(fdir, streams_mask, distance=1)

    # 3. Conversão para GeoDataFrame
    print("Convertendo feições para GeoDataFrame...")
    features = network.get('features', [])
    geometries = []

    if not features:
        print(f"Aviso: Nenhuma feição de canal encontrada com o limiar de {STREAM_THRESHOLD}.")
        canais_path = None
    else:
        for feature in features:
            coords = feature['geometry']['coordinates']
            # Cria LineString apenas se tiver 2 ou mais pontos
            if len(coords) >= 2:
                line = LineString(coords)
                geometries.append(line)

        if not geometries:
            print("Aviso: Nenhuma geometria de linha válida foi criada.")
            canais_path = None
        else:
            # 5. CRIAR E SALVAR O GEODATAFRAME
            streams_gdf = gpd.GeoDataFrame(geometry=geometries, crs=grid.viewfinder.crs)
            canais_path = os.path.join(OUTPUT_DIR, 'rede_drenagem_total.geojson')
            streams_gdf.to_file(canais_path, driver='GeoJSON')
            print(f"Rede de drenagem (total) salva em: '{canais_path}'")

    print("Célula 3 concluída com sucesso!")
    print("\n--- PROCESSO CONCLUÍDO ---")
    print(f"Arquivos gerados em: '{OUTPUT_DIR}'")

    # --- LISTA DE SAÍDA ATUALIZADA ---
    print(f"1. {os.path.join(OUTPUT_DIR, 'flow_accumulation.tif')}")
    print(f"2. {os.path.join(OUTPUT_DIR, 'flow_direction.tif')}")
    print(f"3. {os.path.join(OUTPUT_DIR, 'aspect.tif')}")
    print(f"4. {os.path.join(OUTPUT_DIR, 'slope.tif')}")
    print(f"5. {os.path.join(OUTPUT_DIR, 'twi.tif')}")
    print(f"6. {canais_path if canais_path else '(Nenhum canal gerado)'}")
    # ----------------------------------

    print("\n>>> Carregue estes arquivos no QGIS para encontrar seu ponto de exutório. <<<")
    print("(Dica: Carregue o 'flow_accumulation.tif' e aplique uma rampa de cores 'Invertida' para ver os rios)")

except Exception as e:
    print(f"Erro inesperado na Célula 3: {e}")
    raise


--- Iniciando Célula 3: Vetorização da Rede de Drenagem ---
Criando máscara de rios com limiar > 1000...
Extraindo a rede de drenagem como feições vetoriais... (Pode demorar)
Convertendo feições para GeoDataFrame...
Rede de drenagem (total) salva em: 'resultados_hidrologia_basica\rede_drenagem_total.geojson'
Célula 3 concluída com sucesso!

--- PROCESSO CONCLUÍDO ---
Arquivos gerados em: 'resultados_hidrologia_basica'
1. resultados_hidrologia_basica\flow_accumulation.tif
2. resultados_hidrologia_basica\flow_direction.tif
3. resultados_hidrologia_basica\aspect.tif
4. resultados_hidrologia_basica\slope.tif
5. resultados_hidrologia_basica\twi.tif
6. resultados_hidrologia_basica\rede_drenagem_total.geojson

>>> Carregue estes arquivos no QGIS para encontrar seu ponto de exutório. <<<
(Dica: Carregue o 'flow_accumulation.tif' e aplique uma rampa de cores 'Invertida' para ver os rios)


## Deliação da bacia, Ordem strahler e distancia do fluxo

In [7]:
# %%
# --- CÉLULA 3: Delineamento de Bacia Hidrográfica by MDE
print("\n--- Iniciando Célula 2: Delimitação da Bacia e Canais ---")

# --- !!! DEFINIR COORDENADAS DO EXUTÓRIO AQUI !!! ---
# Substitua pelos valores obtidos visualizando os mapas da Célula 1
ponto_exutorio = (-44.118627, -20.316243) # (Longitude, Latitude) - EXEMPLO
# -----------------------------------------------------

# Verificar se as variáveis da Célula 1 existem
if 'grid' not in locals() or 'fdir' not in locals() or 'acc' not in locals() or 'inflated_dem' not in locals():
    print("Erro: Variáveis da Célula 1 não encontradas. Execute a Célula 1 primeiro.")
    raise NameError("Variáveis da Célula 1 ausentes.")



try:
    print(f"Usando exutório em: {ponto_exutorio}")
    x_outlet, y_outlet = ponto_exutorio

    # Máscara de rios (baseada no limiar global)
    streams_mask = acc > STREAM_THRESHOLD

    # Snap do exutório à rede de drenagem
    print("Realizando snap do exutório...")
    try:
        x_snap, y_snap = grid.snap_to_mask(streams_mask, (x_outlet, y_outlet))
    except ValueError as e:
        print(f"Erro ao fazer snap: {e}. Verifique se as coordenadas estão dentro da área do MDE e próximas a um canal com acumulação > {STREAM_THRESHOLD}.")
        raise

    # Delimitação da Bacia
    print("Delimitação bacia hidrográfica...")
    catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap, xytype='coordinate')

    # CÁLCULO DISTÂNCIA DE FLUXO (ANTES DO RECORTE)
    print("Calculando a distância de fluxo para o exutório (Flow Distance)...")
    dist = grid.distance_to_outlet(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap,
                                   xytype='coordinate')

    # Recorte do grid para a bacia (modifica o objeto 'grid' em memória)
    print("Recortando grid para a bacia...")
    grid.clip_to(catch)
    print("Grid recortado. As próximas operações serão limitadas à bacia.")

    # --- EXPORTAÇÃO DO RASTER DE DISTÂNCIA DE FLUXO (RECORTADO) ---
    dist_view = grid.view(dist)
    dist_path = os.path.join(OUTPUT_DIR, 'flu_distance.tif')
    grid.to_raster(dist_view, dist_path, dtype=rasterio.float32, nodata=-9999.0)
    print(f"Distância de Fluxo (recortada) salva em: '{dist_path}'")

    # ---------------------------------------------------------------------
    # EXTRAÇÃO DA REDE E CÁLCULO STRAHLER (LÓGICA OTIMIZADA)
    # ---------------------------------------------------------------------
    print("Extraindo rede de drenagem e calculando ordem de Strahler (Raster)...")

    # 1. Obter fdir e streams_mask recortadas
    fdir_clipped = grid.view(fdir) # fdir recortado
    # Requer o recalculo da ACC para garantir que a máscara corresponda ao recorte
    clipped_acc = grid.accumulation(fdir_clipped, dirmap=dirmap)
    clipped_streams_mask = clipped_acc > STREAM_THRESHOLD

    # 2. CÁLCULO DO RASTER DE ORDEM DE STRAHLER
    stream_order_raster = grid.stream_order(fdir_clipped, clipped_streams_mask)

    # 3. EXTRAÇÃO DA REDE VETORIAL
    print("Extraindo a rede de drenagem como feições vetoriais...")
    network = grid.extract_river_network(fdir_clipped, clipped_streams_mask, distance=1)

    # Vetorizar Bacia (MANTIDO O MESMO CÓDIGO ANTERIOR)
    print("Vetorizando a bacia...")
    catch_view = grid.view(catch)
    catchment_shapes = rasterio.features.shapes(
        catch_view.astype(np.uint8),
        mask=catch_view,
        transform=grid.viewfinder.affine
    )
    catchment_geoms = [shape(geom) for geom, value in catchment_shapes if value == 1]
    if not catchment_geoms:
        raise RuntimeError("Não foi possível vetorizar a bacia.")

    catchment_gdf_temp = gpd.GeoDataFrame(geometry=catchment_geoms, crs=grid.viewfinder.crs)
    catchment_gdf_temp['id'] = 1
    catchment_gdf = catchment_gdf_temp.dissolve(by='id')

    bacia_path = os.path.join(OUTPUT_DIR, 'bacia.geojson')
    catchment_gdf.to_file(bacia_path, driver='GeoJSON')
    print(f"Bacia salva em: '{bacia_path}'")


    # 4. ATRIBUIÇÃO DO ATRIBUTO (Raster -> Vetor)
    print("Atribuindo ordem de Strahler aos canais vetorizados...")
    features = network.get('features', [])
    geometries = []
    strahler_orders = []

    if not features:
        print("Aviso: Nenhuma feição de canal encontrada na rede extraída.")
        canais_path = None # Indica que não há canais para salvar
    else:
        # Dados do Viewfinder do grid recortado para mapeamento
        vf_affine = grid.viewfinder.affine
        inv_vf_affine = ~vf_affine
        vf_height, vf_width = grid.viewfinder.shape

        for feature in features:
            coords = feature['geometry']['coordinates']
            # Cria LineString apenas se tiver 2 ou mais pontos
            if len(coords) >= 2:
                line = LineString(coords)
                geometries.append(line)

                # Mapeamento do ponto inicial da linha para a célula do raster
                start_coord = coords[0]
                col_frac, row_frac = inv_vf_affine * (start_coord[0], start_coord[1])
                row, col = int(row_frac), int(col_frac)

                order_val = None
                # Verificação de segurança para os índices do array
                if 0 <= row < vf_height and 0 <= col < vf_width:
                    # Lendo o valor de Strahler do raster
                    order_val = stream_order_raster[row, col]

                # Convertendo para inteiro (se não for nodata/None)
                if order_val is not None and not np.isnan(order_val):
                    strahler_orders.append(int(order_val))
                else:
                    strahler_orders.append(None)
            else:
                 print(f"Aviso: Segmento de canal com menos de 2 pontos encontrado e ignorado: {coords}.")


        # 5. CRIAR E SALVAR O GEODATAFRAME COM O ATRIBUTO
        if geometries:
            # Reintroduzindo a coluna 'strahler_order'
            streams_gdf = gpd.GeoDataFrame({'strahler_order': strahler_orders}, geometry=geometries, crs=grid.viewfinder.crs)
            # Nome de arquivo ligeiramente alterado para refletir a otimização
            canais_path = os.path.join(OUTPUT_DIR, 'canais_strahler.geojson')
            streams_gdf.to_file(canais_path, driver='GeoJSON')
            print(f"Canais (com ordem de Strahler - Lógica Otimizada) salvos em: '{canais_path}'")
        else:
            print("Nenhum canal válido para salvar.")
            canais_path = None


    print("Célula 2 concluída com sucesso!")
    print("Variáveis `grid` (recortado), `catch` (máscara) disponíveis.")
    print(f"Arquivos gerados: '{bacia_path}', '{canais_path if canais_path else 'Nenhum canal salvo'}', '{dist_path}'")

except Exception as e:
    print(f"Erro inesperado na Célula 2: {e}")
    raise


--- Iniciando Célula 2: Delimitação da Bacia e Canais ---
Usando exutório em: (-44.118627, -20.316243)
Realizando snap do exutório...
Delimitação bacia hidrográfica...
Calculando a distância de fluxo para o exutório (Flow Distance)...
Recortando grid para a bacia...
Grid recortado. As próximas operações serão limitadas à bacia.
Distância de Fluxo (recortada) salva em: 'resultados_hidrologia_basica\flu_distance.tif'
Extraindo rede de drenagem e calculando ordem de Strahler (Raster)...
Extraindo a rede de drenagem como feições vetoriais...
Vetorizando a bacia...
Bacia salva em: 'resultados_hidrologia_basica\bacia.geojson'
Atribuindo ordem de Strahler aos canais vetorizados...
Canais (com ordem de Strahler - Lógica Otimizada) salvos em: 'resultados_hidrologia_basica\canais_strahler.geojson'
Célula 2 concluída com sucesso!
Variáveis `grid` (recortado), `catch` (máscara) disponíveis.
Arquivos gerados: 'resultados_hidrologia_basica\bacia.geojson', 'resultados_hidrologia_basica\canais_strahl

## HAND = Mancha de inundação

In [9]:
# --- CÉLULA 3: HAND, Raster e Vetor de Inundação ---
print("\n--- Iniciando Célula 3: HAND, Raster e Vetor de Inundação ---")

# Verificar se as variáveis necessárias existem
if 'grid' not in locals() or 'fdir' not in locals() or 'inflated_dem' not in locals() or 'catch' not in locals() or 'acc' not in locals():
    print("Erro: Variáveis das Células 1 ou 2 não encontradas. Execute as células anteriores.")
    raise NameError("Variáveis das Células 1 ou 2 ausentes.")

# As variáveis CHANNEL_DEPTH e STREAM_THRESHOLD são lidas da CÉLULA 1

try:
    # Máscara de rios (baseada no limiar global, usada pelo compute_hand)
    # Usamos o 'acc' total (antes do clip da bacia) para o HAND
    streams_mask_global = acc > STREAM_THRESHOLD

    # HAND (calculado dentro da bacia usando a visão do grid)
    print("Calculando HAND dentro da bacia...")
    hand = grid.compute_hand(fdir, inflated_dem, streams_mask_global)

    # Recorte do HAND para a bacia (aplicando a máscara novamente via grid.view)
    # MODIFICADO: Removemos a referência ao {CENARIO_ATUAL}
    print(f"Calculando mancha de inundação (profundidade) para {CHANNEL_DEPTH}m...")
    hand_view = grid.view(hand, nodata=np.nan) # HAND dentro da bacia

    # Calcula a profundidade da inundação onde HAND < channel_depth
    # Esta é a linha que usa a variável da CÉLULA 1!
    inundation_depth = np.where(hand_view < CHANNEL_DEPTH, CHANNEL_DEPTH - hand_view, np.nan) # Usar NaN como nodata

    # --- Salvar Raster de Inundação ---
    print("Construindo perfil do raster de saída...")
    profile = {
        'crs': grid.viewfinder.crs,
        'transform': grid.viewfinder.affine,
        'height': grid.viewfinder.shape[0],
        'width': grid.viewfinder.shape[1],
        'driver': 'GTiff',
        'count': 1,
        'dtype': rasterio.float32,
        'nodata': np.nan
    }

    # MODIFICADO: Criar um sufixo de nome de arquivo transparente
    # Converte '10.0' para '10_0m' ou '7.5' para '7_5m'
    suffix_nome_arquivo = f"{str(CHANNEL_DEPTH).replace('.', '_')}m"

    # MODIFICADO: Re-habilitando o salvamento do Raster (seu código estava comentado)
    inundacao_raster_path = os.path.join(OUTPUT_DIR, f'inundacao_mapa_{suffix_nome_arquivo}.tif')
    with rasterio.open(inundacao_raster_path, 'w', **profile) as dst:
        dst.write(inundation_depth.astype(rasterio.float32), 1)
    print(f"Arquivo raster de inundação salvo como: '{inundacao_raster_path}'")


    # --- Vetorizar a Mancha de Inundação ---
    print("Vetorizando a mancha de inundação (área)...")
    # Criar uma máscara booleana onde a inundação existe (profundidade não é NaN)
    inundation_mask = ~np.isnan(inundation_depth)

    if not np.any(inundation_mask):
         print("Aviso: Nenhuma área inundada encontrada para vetorizar.")
         inundacao_vetor_path = None
    else:
        # Vetorizar usando a máscara (value será 1 onde a máscara for True)
        shapes_generator = rasterio.features.shapes(
            inundation_mask.astype(np.uint8), # Precisa ser tipo inteiro
            mask=inundation_mask, # Vetoriza apenas onde a máscara é True
            transform=profile['transform'] # Usa a transformação do raster salvo
        )

        # Coletar geometrias onde o valor do pixel vetorizado é 1 (True)
        flood_geometries = [shape(geom) for geom, value in shapes_generator if value == 1]

        if not flood_geometries:
            print("Aviso: Nenhuma geometria encontrada após a vetorização da máscara de inundação.")
            inundacao_vetor_path = None
        else:
            # Criar GeoDataFrame temporário
            flood_gdf_temp = gpd.GeoDataFrame(geometry=flood_geometries, crs=profile['crs'])
            # Adicionar ID para dissolver
            flood_gdf_temp['id'] = 1
            # Dissolver para obter um único MultiPolygon (ou Polygon)
            flood_gdf_dissolved = flood_gdf_temp.dissolve(by='id')

            # Salvar o vetor GeoJSON
            # MODIFICADO: Nome do arquivo agora é transparente
            inundacao_vetor_path = os.path.join(OUTPUT_DIR, f'inundacao_{suffix_nome_arquivo}.geojson')
            flood_gdf_dissolved.to_file(inundacao_vetor_path, driver='GeoJSON')
            print(f"Arquivo vetorial de inundação salvo como: '{inundacao_vetor_path}'")

    print(f"Célula 3 concluída com sucesso!")
    # MODIFICADO: Corrigindo o print final
    print(f"Variáveis `inundacao_raster_path` e `inundacao_vetor_path` disponíveis.")

except Exception as e:
    print(f"Erro inesperado na Célula 3: {e}")
    raise


--- Iniciando Célula 3: HAND, Raster e Vetor de Inundação ---
Calculando HAND dentro da bacia...
Calculando mancha de inundação (profundidade) para 10.0m...
Construindo perfil do raster de saída...
Arquivo raster de inundação salvo como: 'resultados_hidrologia_basica\inundacao_mapa_10_0m.tif'
Vetorizando a mancha de inundação (área)...
Arquivo vetorial de inundação salvo como: 'resultados_hidrologia_basica\inundacao_10_0m.geojson'
Célula 3 concluída com sucesso!
Variáveis `inundacao_raster_path` e `inundacao_vetor_path` disponíveis.


## DOWNLOAD OSMNX

In [ ]:
# %%
# --- CÉLULA 4: Código Completo e Corrigido ---
# ASSUMIR IMPORTS: import os, import geopandas as gpd, from shapely.geometry import box, import osmnx as ox, from pyproj import CRS
# Nota: É crucial que CRS (de pyproj) esteja importado se a variável CRS não estiver disponível.

print("\n--- Iniciando Célula 4: Download de Dados OSM (Múltiplas Tags) ---")

# --- DEFINE CAMINHO DO NOVO ARQUIVO DE ENTRADA (GeoJSON) ---
# Usamos os.path.join para lidar corretamente com barras e contrabarras em diferentes SOs
inundacao_geojson_path = os.path.join('resultados_hidrologia_basica', 'inundacao.geojson')

# Verificar se o arquivo GeoJSON existe
if not os.path.exists(inundacao_geojson_path):
    print(f"Erro: Caminho do GeoJSON de inundação não encontrado em '{inundacao_geojson_path}'.")
    raise NameError("Arquivo GeoJSON de inundação ausente ou inválido.")

# Define os nomes dos arquivos de saída
osm_polygons_path = os.path.join(OUTPUT_DIR, 'osm_polygons.gpkg')
osm_lines_path = os.path.join(OUTPUT_DIR, 'osm_lines.gpkg')
osm_points_path = os.path.join(OUTPUT_DIR, 'osm_points.gpkg')

try:
    # --- Definir AOI a partir do GeoJSON de Inundação ---
    print(f"Lendo GeoJSON '{inundacao_geojson_path}' para definir AOI...")

    # Leitura com geopandas
    src_gdf = gpd.read_file(inundacao_geojson_path)

    # Extrair o CRS e o Bounding Box (Envelope) total dos dados
    flood_crs = src_gdf.crs
    flood_bounds = src_gdf.total_bounds

    # Cria o Bounding Box (AOI retangular) na CRS nativa dos dados
    aoi_bbox_native = box(*flood_bounds)
    aoi_gdf_native = gpd.GeoDataFrame([1], geometry=[aoi_bbox_native], crs=flood_crs)

    # Reprojetar para EPSG:4326 (necessário para OSMnx)
    target_crs_epsg = 4326
    if flood_crs.to_epsg() != target_crs_epsg:
        print(f"Reprojetando AOI de {flood_crs.to_string()} para EPSG:{target_crs_epsg}...")
        aoi_gdf_4326 = aoi_gdf_native.to_crs(epsg=target_crs_epsg)
    else:
        print("AOI já está em EPSG:4326.")
        aoi_gdf_4326 = aoi_gdf_native

    # Correção do erro de referência 'ai_gdf_4326'
    aoi_polygon_4326 = aoi_gdf_4326.geometry.iloc[0]
    print(f"AOI definida e em EPSG:{target_crs_epsg}.")

    # --- Coletar Dados OSM (Múltiplas Tags) ---
    print("Baixando dados do OpenStreetMap (Máximo de Tags)...")

    # Define as tags a serem buscadas (LISTA ABRANGENTE)
    tags_to_fetch = {
        # 1. Edificações e Estruturas
        'building': True, 'building:part': True, 'roof': True, 'man_made': True,

        # 2. Vias de Transporte
        'highway': True, 'railway': True, 'aeroway': True, 'public_transport': True,

        # 3. Recursos Hídricos e Naturais
        'natural': True,
        'waterway': True,
        'landuse': True,

        # 4. Comodidades e Serviços
        'amenity': True, 'shop': True, 'office': True, 'leisure': True, 'tourism': True, 'historic': True,

        # 5. Infraestrutura e Utilidades
        'power': True, 'telecom': True, 'pipeline': True,

        # 6. Fronteiras (Mantido para pegar limites específicos, mas será filtrado em seguida)
        'boundary': True,

        # 7. Defesas Contra Inundação (Tags Focadas)
        'wall': ['flood_wall', 'retaining_wall'], 'barrier': True, 'flood_asset': True,

        # 8. Áreas de Risco e Vulnerabilidade (Tags Focadas)
        'flood_prone': ['yes'],

        # 9. Locais de Emergência (Tags Focadas)
        'emergency': True,

        # Adiciona 'place' para o filtro funcionar, se esta tag estiver trazendo cidades
        'place': True
    }

    # Correção do erro 'AttributeError: features_from_polygon' (box -> ox)
    osm_features_gdf = ox.features_from_polygon(aoi_polygon_4326, tags_to_fetch)
    print(f"Download concluído. {len(osm_features_gdf)} feições encontradas inicialmente (tipos mistos).")

    # Certificar-se que o CRS está definido
    if osm_features_gdf.crs is None:
         osm_features_gdf.crs = CRS.from_epsg(target_crs_epsg)

    # --- NOVO PASSO: Remover Feições Administrativas/Locais (Cidades e Regiões) ---
    print("Filtrando feições administrativas e de assentamento (boundary/place)...")

    keys_to_exclude = ['boundary', 'place']
    initial_count = len(osm_features_gdf)

    # Cria uma máscara para identificar as linhas a serem EXCLUÍDAS
    exclusion_mask = False
    for key in keys_to_exclude:
        # A GeoDataFrame pode não ter a coluna se a tag não foi encontrada
        if key in osm_features_gdf.columns:
            # Acumula True para CADA feição que TEM a tag indesejada (ou seja, valor não é NaN)
            exclusion_mask |= osm_features_gdf[key].notna()

    # Aplica a máscara: mantemos apenas o que está FORA da máscara de exclusão (~)
    osm_features_gdf = osm_features_gdf[~exclusion_mask].copy()

    print(f"Feições removidas: {initial_count - len(osm_features_gdf)}. Feições restantes: {len(osm_features_gdf)}.")

    # -----------------------------------------------------------------------
    # --- 1. Filtrar e Salvar POLÍGONOS ---
    osm_polygons_gdf = osm_features_gdf[
        osm_features_gdf.geometry.notna() &
        osm_features_gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])
    ].copy()

    if not osm_polygons_gdf.empty:
        osm_polygons_gdf.to_file(osm_polygons_path, driver="GPKG")
        print(f"Polígonos salvos em: '{osm_polygons_path}' ({len(osm_polygons_gdf)} feições)")
    else:
        print("Aviso: Nenhuma feição poligonal encontrada. Arquivo de polígonos não salvo.")
        osm_polygons_path = None

    # --- 2. Filtrar e Salvar LINHAS ---
    osm_lines_gdf = osm_features_gdf[
        osm_features_gdf.geometry.notna() &
        osm_features_gdf.geometry.type.isin(['LineString', 'MultiLineString'])
    ].copy()

    if not osm_lines_gdf.empty:
        osm_lines_gdf.to_file(osm_lines_path, driver="GPKG")
        print(f"Linhas salvas em: '{osm_lines_path}' ({len(osm_lines_gdf)} feições)")
    else:
        print("Aviso: Nenhuma feição de linha encontrada. Arquivo de linhas não salvo.")
        osm_lines_path = None

    # --- 3. Filtrar e Salvar PONTOS ---
    osm_points_gdf = osm_features_gdf[
        osm_features_gdf.geometry.notna() &
        osm_features_gdf.geometry.type == 'Point'
    ].copy()

    if not osm_points_gdf.empty:
        osm_points_gdf.to_file(osm_points_path, driver="GPKG")
        print(f"Pontos salvos em: '{osm_points_path}' ({len(osm_points_gdf)} feições)")
    else:
        print("Aviso: Nenhuma feição de ponto encontrada. Arquivo de pontos não salvo.")
        osm_points_path = None

    print(f"\nCélula 4 concluída com sucesso! Todos os tipos de geometria foram processados.")

except Exception as e:
    print(f"Erro inesperado na Célula 4: {e}")
    raise